In [5]:
import os
import numpy as np
import pandas as pd
from time import time
from utils import custom_preprocessing, dependencies

data_dir = "/home/stavros/DATA/AirbnbReviews"
#data_dir = "D:/"

In [6]:
area = "south_aegean"
area_dir = os.path.join(data_dir, area)

island_samples = {"Santorini": 100266, "Mykonos": 40866, "Rhodes": 25566}

island_reviews = {k: pd.read_pickle(os.path.join(
    area_dir, "reviews_with_aspects_{}_{}samples_sentiment.pkl".format(k, n)))
                  for k, n in island_samples.items()}

for k, r in island_reviews.items():
    print("{}: {}".format(k, r.shape))

print()
# Check how many reviews have no identified aspects (percentage)
for k, r in island_reviews.items():
    print("{}: {}".format(k, pd.isnull(r["aspects"]).sum() * 100.0 / len(r)))

print()
# Clean reviews
for k, r in island_reviews.items():
    island_reviews[k] = r[pd.notnull(r["aspects"])]
    print("{}: {}".format(k, island_reviews[k].shape))

Santorini: (100266, 10)
Mykonos: (40866, 10)
Rhodes: (25566, 10)

Santorini: 16.34751560848144
Mykonos: 16.926050995937942
Rhodes: 28.737385590237032

Santorini: (83875, 10)
Mykonos: (33949, 10)
Rhodes: (18219, 10)


In [7]:
island_reviews["Rhodes"].columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments',
       'location', 'processed_comments', 'aspects', 'aspects_sentiment'],
      dtype='object')

In [8]:
location_sentiment = island_reviews["Santorini"]["aspects_sentiment"].map(lambda x: x["location"] if "location" in x else 0)

In [12]:
santorini_loc_data = island_reviews["Santorini"][location_sentiment != 0]
santorini_loc_data["location_sentiment"] = location_sentiment[location_sentiment != 0]
print(santorini_loc_data.shape)

(20615, 11)


/home/stavros/PythonEvironments/neuralcoref-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
unique_ids = np.unique(santorini_loc_data["listing_id"])
listing_loc_sentiment = {}
for ids in unique_ids:
    listing_data = santorini_loc_data[santorini_loc_data["listing_id"] == ids]
    ranks, counts = np.unique(listing_data["location_sentiment"], return_counts=True)
    listing_loc_sentiment[ids] = [(times, rank) for rank, times in zip(ranks, counts)]

In [25]:
listing_loc_score = np.array([sum(times * (2 * int(rank > 0) - 1) for times, rank in listing_loc_sentiment[k]) 
                              for k in unique_ids])